In [ ]:
from deepgtav.messages import Start, Stop, Scenario, Commands, frame2numpy
from deepgtav.client import Client

import argparse
import time
import cv2

In [ ]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model,  Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing import image
from keras.applications import VGG16
from keras.layers import Input, Lambda, Cropping2D, Activation, ELU
from keras.layers.merge import add, concatenate
from keras.models import model_from_json

%matplotlib inline

In [ ]:
results_path = 'D:/ML_DATA/wheelai/gtav/results/'

In [ ]:
def get_model(model, weights):
    
    model_path = results_path + model
    weights = results_path + weights
    # load model
    json_file = open(model_path, 'r')
    model_json = json_file.read()
    json_file.close()
    model = model_from_json(model_json)
    # load weights into new model
    model.load_weights(weights)
    print("Loaded model from disk")
    # compile model
    model.compile(optimizer='adam',loss='mse')
    return model

In [ ]:
model = get_model('commaai.json', 'commaai1.h5')

In [ ]:
client = Client(ip='localhost', port=8000)

# We set the scenario to be in manual driving, and everything else random (time, weather and location). 
# See deepgtav/messages.py to see what options are supported
scenario = Scenario(drivingMode=-1) #manual driving

# Send the Start request to DeepGTAV. Dataset is set as default, we only receive frames at 10Hz (320, 160)
client.sendMessage(Start(scenario=scenario))

In [ ]:
def predict(img):
    
    #x = image.img_to_array(img)
    x = img[80:,:,:]
    x = np.expand_dims(x, axis=0)
    return model.predict(x, batch_size=1,verbose=0)[0].tolist()

In [ ]:
stoptime = time.time() + 80*3600
while time.time() < stoptime:
        try:
            # We receive a message as a Python dictionary
            message = client.recvMessage()	

            # The frame is a numpy array that can we pass through a CNN for example		
            image = frame2numpy(message['frame'], (320,160))
            commands = predict(image)
            print (commands)
            # We send the commands predicted by the agent back to DeepGTAV to control the vehicle
            client.sendMessage(Commands(commands[0], commands[1], commands[2]))
            #client.sendMessage(Commands(1, 0, 0))
            #print('ok')
        except KeyboardInterrupt:
            break
    
    # We tell DeepGTAV to stop
client.sendMessage(Stop())
print('stopped')
client.close()
print('disconnected')